In [1]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [3]:
# Sample resumes
resumes = {
    "Resume A": "I have experience in Python, machine learning, and data analysis. I built ML models and worked with Pandas and NumPy.",
    "Resume B": "Skilled in SQL and Excel. I created dashboards in Tableau and generated reports for business analysis.",
    "Resume C": "Worked with deep learning models like BERT and Transformers. I'm passionate about NLP and language models."
}


In [4]:
# Sample job descriptions
job_descriptions = {
    "JD 1": "Looking for a Python developer with experience in machine learning and data analysis.",
    "JD 2": "Need a data analyst who knows SQL, Excel, and can create dashboards using BI tools like Tableau.",
    "JD 3": "Hiring NLP intern familiar with BERT, Transformers, and deep learning frameworks."
}


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
resume_texts = list(resumes.values())
jd_texts = list(job_descriptions.values())
resume_embeddings = model.encode(resume_texts, convert_to_tensor=True)
jd_embeddings = model.encode(jd_texts, convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
similarity_matrix = util.cos_sim(resume_embeddings, jd_embeddings).cpu().numpy()

In [8]:
similarity_df = pd.DataFrame(similarity_matrix * 100, index=resumes.keys(), columns=job_descriptions.keys())
print("🔍 Similarity Scores (in %):")
print(similarity_df.round(2))

🔍 Similarity Scores (in %):
               JD 1       JD 2       JD 3
Resume A  70.570000  41.480000  33.660000
Resume B  43.680000  84.889999  24.690001
Resume C  35.650002  15.770000  76.419998


In [9]:
print("\n✅ Best JD match for each resume:")
for resume in similarity_df.index:
    best_jd = similarity_df.loc[resume].idxmax()
    score = similarity_df.loc[resume, best_jd]
    print(f"{resume} ➡️ {best_jd} (Score: {score:.2f}%)")



✅ Best JD match for each resume:
Resume A ➡️ JD 1 (Score: 70.57%)
Resume B ➡️ JD 2 (Score: 84.89%)
Resume C ➡️ JD 3 (Score: 76.42%)
